In [85]:
# import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# import csv file and save as dataframe
weather_df = pd.read_csv("uber_nyc_enriched.csv")
weather_df.head()

,pickup_dt,borough,pickups,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday
0,2015-01-01 01:00:00,Bronx,152,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
1,2015-01-01 01:00:00,Brooklyn,1519,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
2,2015-01-01 01:00:00,EWR,0,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
3,2015-01-01 01:00:00,Manhattan,5258,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
4,2015-01-01 01:00:00,Queens,405,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y


In [86]:
# rename columns
weather_df = weather_df.rename(columns = {"pickup_dt": "Pickup Date",
                                          "borough": "Borough",
                                          "pickups": "Pickups",
                                          "spd": "Wind Speed (MPH)",
                                          "vsb": "Visibility (Miles)",
                                          "temp": "Temperature (F)",
                                          "dewp": "Dewpoint",
                                          "slp": "Sea Level Pressure",
                                          "pcp01": "1 Hour Precipitation",
                                          "pcp06": "6 Hour Precipitation",
                                          "pcp24": "24 Hour Precipitation",
                                          "sd": "Snow Depth (inches)",
                                          "hday": "Holiday"})
weather_df.head()

,Pickup Date,Borough,Pickups,Wind Speed (MPH),Visibility (Miles),Temperature (F),Dewpoint,Sea Level Pressure,1 Hour Precipitation,6 Hour Precipitation,24 Hour Precipitation,Snow Depth (inches),Holiday
0,2015-01-01 01:00:00,Bronx,152,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
1,2015-01-01 01:00:00,Brooklyn,1519,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
2,2015-01-01 01:00:00,EWR,0,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
3,2015-01-01 01:00:00,Manhattan,5258,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
4,2015-01-01 01:00:00,Queens,405,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y


In [76]:
# make pickup dataframe based on bad wind

# arbitrary cutoffs for inclement weather: wind > 20 mph, temp < 40 or temp > 85, 24 hour precip >= 1, snow depth >= 2
bad_wind_df = weather_df.dropna()

bad_wind_df = bad_wind_df.loc[(bad_wind_df["Wind Speed (MPH)"] >= 20)]

bad_wind_df = bad_weather_df.dropna()
len(bad_wind_df)

18618

In [77]:
# create bad temperature dataframes
bad_hitemp_df = weather_df.dropna()
bad_lotemp_df = weather_df.dropna()

bad_hitemp_df = bad_hitemp_df.loc[(bad_hitemp_df["Temperature (F)"] >= 85)]
bad_lotemp_df = bad_lotemp_df.loc[(bad_lotemp_df["Temperature (F)"] <= 40)]

# append high temp and low temp dataframes creating bad temp dataframe
bad_temp_df = bad_hitemp_df.append(bad_lotemp_df)
len(bad_temp_df)

11298

In [78]:
# create bad precipitation dataframe
bad_precip_df = weather_df.dropna()

bad_precip_df = bad_precip_df.loc[(bad_precip_df["24 Hour Precipitation"] >= 1)]
len(bad_precip_df)

318

In [79]:
# create bad snow weather dataframe
bad_snow_df = weather_df.dropna()

bad_snow_df = bad_snow_df.loc[(bad_snow_df["Snow Depth (inches)"] >= 2)]
len(bad_snow_df)

6966

In [82]:
# append all the bad weather dataframes
bad_weather_df = bad_wind_df.append([bad_temp_df, bad_precip_df, bad_snow_df])
len(bad_weather_df)

37200